In [30]:
import numpy as np
import json
import os
import sys
import random
import torch

In [4]:
'SPOT-2', 'SPOT-3', 'SPOT-4', 'Spot-1', 'fire hydrant', 'invalid spot - entrance', 'invalid spot - fire hydrant', 'invalid spot - white curb', 'invalid spot-red curb', 'invalid spot-yellow curb', 'parking-spots', 'spot _1', 'spot four cars', 'spot three cars'

('SPOT-2',
 'SPOT-3',
 'SPOT-4',
 'Spot-1',
 'fire hydrant',
 'invalid spot - entrance',
 'invalid spot - fire hydrant',
 'invalid spot - white curb',
 'invalid spot-red curb',
 'invalid spot-yellow curb',
 'parking-spots',
 'spot _1',
 'spot four cars',
 'spot three cars')

In [5]:
label_conf = {'SPOT-2':1,
 'SPOT-3':2,
 'SPOT-4':3,
 'Spot-1':0,
 'fire hydrant':7,
 'invalid spot - entrance':6,
 'invalid spot - fire hydrant':4,
 'invalid spot - white curb':5,
 'invalid spot-red curb':5,
 'invalid spot-yellow curb':5,
 'parking-spots':0,
 'spot _1':1,
 'spot four cars':3,
 'spot three cars':2}

In [6]:
def change_label_config(filePath):
    with open(filePath,'r') as file:
        label_config = json.load(file)
    for key, value in label_config.items():
        if value == 1:
            label_config[key] = 0
    for key, value in label_config.items():
        if value == 3 or value == 2:
            label_config[key] = 1
    for key, value in label_config.items():
        if value == 4 :
            label_config[key] = 2
    for key, value in label_config.items():
        if value == 5:
            label_config[key] = 3
    return label_config

In [7]:
def change_aux(filePath):
    with open(filePath,'r') as file:
        label_config = json.load(file)
    for key, value in label_config.items():
        if value == -1:
            label_config[key] = 0
    return label_config

In [96]:
with open(label_conf_path,"r") as file:
    a = json.load(file)

In [120]:
for label_conf_path in configs:
    new_conf = change_aux(label_conf_path)
    with open(label_conf_path,'w') as f:
        f.write(json.dumps(new_conf))

In [8]:
def delete_label(label_file, label):
    size = os.path.getsize(label_file)
    lines = []
    if size == 0:
        return None
    else:
        with open(label_file,'r') as f:
            for line in f.readlines():
                idx = line.index(" ")
                if int(line[:idx]) == label:
                    pass
                else:
                    lines.append(line)
        with open(label_file, 'w') as f:
            for line in lines:
                f.write(line)

In [9]:
def count_background(folder,mode):
    path = f'{folder}/{mode}/labels/'
    labels = os.listdir(path)
    backgroundcount = 0
    for label in labels:
        filesize = os.path.getsize(path+label)
        if filesize == 0:
            backgroundcount += 1
    return backgroundcount/len(labels), len(labels)

In [9]:
count_background("data/merged_data/","train")

(0.10123042505592841, 10728)

In [12]:
def remove_background_images(folder,mode,proportion = 0.1,random_state=42):
    path_images = f'{folder}{mode}/images/'
    path_labels = f'{folder}{mode}/labels/'
    labels = [path_labels + label for label in os.listdir(path_labels)]

    background_images = []
    for label in labels:
        if os.path.getsize(label) == 0:
            background_images.append(label)
    if len(background_images)/len(labels) < proportion:
        return None
    else:
        to_remove = int( len(background_images) - proportion*len(labels) )
        random.seed(random_state)
        choices = random.choices(background_images, k= to_remove)
        for path_label in choices : 
            path_img = path_label.replace(f"{mode}/labels/",f"{mode}/images/")
            path_img = f"{path_img[:-4]}.jpg"
            #print(os.path.isfile(path_img) ,os.path.isfile(path_label))
            if os.path.isfile(path_img) and os.path.isfile(path_label):
                os.remove(path_label)
                os.remove(path_img)
    return count_background(folder,mode)   
            



In [13]:
import shutil
def check_arborescence(folder):
    try:
        shutil.rmtree(folder+'/valid/')
    except FileNotFoundError:
        pass
    try:
        shutil.rmtree(folder+'/test/')
    except FileNotFoundError:
        pass
    os.mkdir(folder+'/valid/')
    os.mkdir(folder+'/test/')
    os.mkdir(folder+'/valid/images/')
    os.mkdir(folder+'/valid/labels/')
    os.mkdir(folder+'/test/images/')
    os.mkdir(folder+'/test/labels/')
            

In [14]:
len(os.listdir('data/merged_data/train/labels/'))

24185

In [18]:
import subprocess
import random
def train_test_resplit(folder,mode='valid',size=0.3,random_state=42):
    ## write an rsync commands to merge the directories
    check_arborescence(folder)
    images = os.listdir(f'{folder}/train/images')
    labels = os.listdir(f'{folder}/train/labels')
    random.seed(random_state)
    val_elements = random.choices(images,k=int(size*len(images))) 
    for elm in val_elements:
        path_img, path_label = f"{folder}/train/images/{elm}", f"{folder}/train/labels/{elm[:-4]}.txt"
        try:
            shutil.move(f"{folder}/train/images/{elm}", f"{folder}/{mode}/images/{elm}")
            shutil.move(f"{folder}/train/labels/{elm[:-4]}.txt", f"{folder}/{mode}/labels/{elm[:-4]}.txt")
        except FileNotFoundError:
            print('Not found ?? ')
            

In [20]:
PATH = "data/final_data_backup/"
folders = os.listdir(PATH)
for folder in folders:
    if '.ipynb_checkpoints' in folder:
        continue
    train_test_resplit(PATH+folder,mode="valid",size=0.1)
    train_test_resplit(PATH+folder,mode="test",size=0.1)

Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not found ?? 
Not fo

In [21]:
class UnifyLabels :
    def __init__(self, folderPath, labelConfig):
        self.folderPath = folderPath
        self.labelConfig = labelConfig
        self.labelEncoder = self.generate_label_encoder()
        self.labelEncoder[-1] = 0
    
    def get_labels(self):
        labels = []
        start = len("names: ")
        with open(self.folderPath+'/data.yaml','r') as file:
            for line in file.readlines():
                if "names: " in line:
                    labels += eval(line[start :])
                    break
        return labels
    
    def generate_label_encoder(self):
        "label config should assign a integer to every present label"
        labels = self.get_labels()
        res = {}
        for n, label in enumerate(labels):
            res[n] = self.labelConfig[label]
        return res
    
    def relabel_file(self, filePath):
        if not os.path.isdir(filePath):
            lines = []
            with open(filePath,'r') as f:
                for line in f.readlines():
                    idx = line.index(" ")
                    newClass = self.labelEncoder[int(line[:idx])]
                    lines.append( f'{newClass}' + line[idx:])
            with open(filePath, 'w') as f:
                for line in lines:
                    f.write(line)
    def delete_label(self, filePath,label):
        size = os.path.getsize(filePath)
        lines = []
        if size == 0:
            return None
        else:
            with open(filePath,'r') as f:
                for line in f.readlines():
                    idx = line.index(" ")
                    if int(line[:idx]) == label:
                        pass
                    else:
                        lines.append(line)
            with open(filePath, 'w') as f:
                for line in lines:
                    f.write(line)
    
    
    def relabel_folder(self):
        for mode in ['train','test','valid']:
            listFiles = [self.folderPath + f'/{mode}/labels/{file}' for file in os.listdir(self.folderPath + f'/{mode}/labels/') ]
            for file in listFiles:
                if ".ipynb_checkpoints" in file:
                    continue
                self.relabel_file(file)
    def delete_label_from_folder(self,labels_to_delete):
        for mode in ['train','test','valid']:
            listFiles = [self.folderPath + f'/{mode}/labels/{file}' for file in os.listdir(self.folderPath + f'/{mode}/labels/') ]
            for file in listFiles:
                for label in labels_to_delete:
                    self.delete_label(label)

In [22]:
import re
path = "data/final_data_backup/"
number_reg = re.compile('\d+')
folders = os.listdir(path)
for folder in folders:
    if '.ipynb_checkpoints' in folder:
        continue
    number = number_reg.findall(folder)[0]
    #print(number)
    with open(f'data/batch_configs/label_config_{number}.txt','r') as file:
        label_config = json.load(file)
    unif = UnifyLabels(path+folder, label_config)
    unif.relabel_folder()

In [49]:
unif.labelConfig

{'fireHydrant': 5,
 'invalidSpot-entrance': 4,
 'invalidSpot-fireHydrant': 2,
 'invalidSpot-redCurb': 3,
 'invalidSpot-whiteCurb': 3,
 'invalidSpot-yellowCurb': 3,
 'spot-1': 0,
 'spot-2': 1,
 'spot-3': 1,
 'spot-4': 1}

In [23]:
len(os.listdir("data/merged_data/train/images"))

40595

In [13]:
def get_labels(filePath):
    labels = []
    start = len("names: ")
    with open(filePath,'r') as file:
        for line in file.readlines():
            if "names: " in line:
                labels += eval(line[start :])
                break
    return labels
def generate_label_dictionary(filePath, labelConfig):
    "label config should assign a integer to every present label"
    labels = get_labels(filePath)
    res = {}
    for n, label in enumerate(labels):
        res[n] = labelConfig[label]
    return res
        

In [134]:

path = f"data/merged_data/train/labels/"
files =  [path + elm for elm in os.listdir(path) if not os.path.isdir(path+elm)]
for file in files:
            with open(file,'r') as f:
                for line in f.readlines():
                    idx = line.index(" ")
                    assert (int(line[:idx]) < 4)

In [79]:
file

'data/merged_data/train/labels/538697340832571_jpg.rf.aa248eebccc5effcc7bc58e3c23ce937.txt'

In [133]:
for i in range(1,26):
    if i != 6:
        path = f"data/final_data_4_cls/batch_{i}/train/labels/"
        files =  [path + elm for elm in os.listdir(path) if not os.path.isdir(path+elm)]
        for file in files:
            with open(file,'r') as f:
                for line in f.readlines():
                    idx = line.index(" ")
                    assert (int(line[:idx]) < 4)

In [139]:
with open(file,'r') as f:
    for line in f.readlines():
        print(line)

In [36]:
remove_background_images("data/merged_data/","train",proportion = 0.10,random_state=42)

(0.10013644183425283, 16857)